In [1]:
%pip install python-dotenv # Install the missing module 'dotenv'
%pip install datasets

Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.5/417.5 kB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 34.6 MB/s eta 0:00:00
  Attempting uninstall: dill
    Found existing installation: dill 0.3.7
    Uninstalling dill-0.3.7:
      Successfully uninstalled dill-0.3.7
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from dotenv import load_dotenv
from ibm_cloud_sdk_core import IAMTokenManager
from ibm_watson_studio_lib import access_project_or_space

wslib = access_project_or_space({
        'token': 'p-2+kQvRm/s+j4gr3kN/L8nE9w==;jo3EuWo2iO+1+F0egvXSew==:ibH3fFKnohJx8UOS9q9Vf+Ewhuye4ERRlqD3CqbrUpUQvU3alsEPblep77PGbqbFrl+zlVsvRbeoVTX4TtP803SeBdM9yhyA2Q==',
        'project_id': '512eaa42-cac0-46c4-a80c-d138ac7ccccc'
})

wslib.download_file('config.env')
load_dotenv('config.env')

# Connection variables
api_key = os.getenv("API_KEY", None)
ibm_cloud_url = os.getenv("IBM_CLOUD_URL", None) 
project_id = os.getenv("PROJECT_ID", None)
creds = {
    "url": ibm_cloud_url,
    "apikey": api_key 
}
access_token = IAMTokenManager(
    apikey = api_key,
    url = "https://iam.cloud.ibm.com/identity/token"
).get_token()

In [3]:
# 2- Load the dataset:

from datasets import load_dataset
dataset = load_dataset("OussamaAzz/sql_dataset_cleaned")
# pd_dataset = pd.DataFrame(dataset)
# pd_dataset.head()
dataset,'\n',dataset['train'][0]

Generating train split:   0%|          | 0/8244 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/825 [00:00<?, ? examples/s]

(DatasetDict({
     train: Dataset({
         features: ['text', 'source'],
         num_rows: 8244
     })
     validation: Dataset({
         features: ['text', 'source'],
         num_rows: 825
     })
 }),
 '\n',
 {'text': '### QUESTION\nwhere pageant is elite model look and year is bigger than 1993.0, who is the delegate?\n\n### CONTEXT\nCREATE TABLE table_1825751_14 (delegate VARCHAR, pageant VARCHAR, year VARCHAR)\n\n### ANSWER\nSELECT delegate FROM table_1825751_14 WHERE pageant = "Elite Model Look" AND year > 1993.0</s>',
  'source': {'answer': 'SELECT delegate FROM table_1825751_14 WHERE pageant = "Elite Model Look" AND year > 1993.0',
   'context': 'CREATE TABLE table_1825751_14 (delegate VARCHAR, pageant VARCHAR, year VARCHAR)',
   'question': 'where pageant is elite model look and year is bigger than 1993.0, who is the delegate?'}})

In [4]:
# import json

# def convert_to_json(dataset):
#     json_data = []
#     for example in dataset['train']['source']:
#         json_data.append({
#             'output': f"### ANSWER\n{example['answer']}",
#             'input': f"### QUESTION\n{example['question']}\n\n### CONTEXT\n{example['context']}",
#             'instruction':f"### CONTEXT\n{example['context']}"
#         })
#     return json_data

# json_data = convert_to_json(dataset)
# with open('sql_dataset.json', 'w') as f:
#     json.dump(json_data, f, indent=4)

In [5]:
!ls

config.env


In [29]:
import json
from ibm_watson_studio_lib import access_project_or_space

# Assuming your JSON file is named "sql_dataset.json"
with open("sql_dataset.json", "r") as f:
    json_data = json.load(f)


# Upload the JSON file to your project as a data asset
wslib.upload_file(
    file_path="sql_dataset.json",
    asset_name="sql_dataset.json",  # You can customize the name
    overwrite=True  # Overwrite existing file with the same name (optional)
)

print("Successfully uploaded sql_dataset.json to your project!")

Successfully uploaded sql_dataset.json to your project!


In [6]:
from datasets import load_dataset
import json

# Load the original dataset
dataset = load_dataset("b-mc2/sql-create-context")
wslib.download_file('sql_dataset_cleaned.json')
# Load your existing dataset to avoid duplicates
with open('sql_dataset_cleaned.json', 'r') as f:
    existing_dataset = json.load(f)

# Create a set of existing items (assuming each item is a dictionary and can be converted to a frozenset)
existing_set = set(frozenset(item.items()) for item in existing_dataset)

# Filter the original dataset to find new unique samples
unique_samples = []
for sample in dataset['train']:
    sample_frozenset = frozenset(sample.items())
    if sample_frozenset not in existing_set:
        unique_samples.append(sample)
    if len(unique_samples) == 500:
        break

# Save the unique samples to a new JSON file
with open('unique_sql_samples.json', 'w') as f:
    json.dump(unique_samples, f, indent=4)

print(f"Saved {len(unique_samples)} unique samples to 'unique_sql_samples.json'")


Generating train split:   0%|          | 0/78577 [00:00<?, ? examples/s]

Saved 500 unique samples to 'unique_sql_samples.json'


In [7]:
!ls

config.env  sql_dataset_cleaned.json  unique_sql_samples.json


In [8]:
# Upload the JSON file to your project as a data asset
wslib.upload_file(
    file_path="unique_sql_samples.json",
    asset_name="unique_sql_samples.json",  # You can customize the name
    overwrite=True  # Overwrite existing file with the same name (optional)
)

print("Successfully uploaded sql_dataset.json to your project!")

Successfully uploaded sql_dataset.json to your project!


In [ ]:
import pandas as pd
from datasets import Dataset

# Your input data
input_data = [
    {
        "output": "SELECT delegate FROM table_1825751_14 WHERE pageant = \"Elite Model Look\" AND year > 1993.0",
        "input": "CREATE TABLE table_1825751_14 (delegate VARCHAR, pageant VARCHAR, year VARCHAR)",
        "instruction": "where pageant is elite model look and year is bigger than 1993.0, who is the delegate?"
    }
    # Add more items as needed
]

# Function to transform data
def transform_to_qca(input_data):
    transformed_data = []

    for item in input_data:
        question = item["instruction"]
        context = item["input"]
        answer = item["output"]

        qca_text = f"### QUESTION\n{question}\n\n### CONTEXT\n{context}\n\n### ANSWER\n{answer}"
        qca_source = {
            "question": question,
            "context": context,
            "answer": answer
        }

        transformed_data.append({
            "text": qca_text,
            "source": qca_source
        })

    return transformed_data

# Transform the input data
transformed_data = transform_to_qca(input_data)

# Create a single Dataset
dataset = Dataset.from_pandas(pd.DataFrame(transformed_data))

# Save the dataset
dataset.save_to_disk("path_to_save_dataset")

# Upload to Hugging Face
from datasets import load_from_disk

dataset = load_from_disk("path_to_save_dataset")
dataset.push_to_hub("your-username/my-dataset")
